# Keypoints imagenes CaVFish Colombia 

# HRNet

#### Organizar archivo json a partir del xml de CVAT

In [ ]:
###
#Se extrae la etiqueta de la forma del pez y 20 keypoints para 1758 spp. a partir de xml (CVAT)
##

import json
import numpy as np
from lxml import etree
from pathlib import Path
from tqdm import tqdm

# === CONFIGURA TUS RUTAS AQUÍ ===
xml_folder_path = Path("/data/Datasets/Fish/CavFish/xml/")  # Carpeta que contiene los XML
output_path = Path("/data/Datasets/Fish/CavFish/fisk_all_20kpts_2025.json")
padding = 10  # píxeles extra para las bboxes

# === INICIALIZAR ESTRUCTURA COCO ===
coco_output = {
    "images": [],
    "annotations": [],
    "categories": [{
        "id": 1,
        "name": "fish",
        "keypoints": [str(i) for i in range(1, 21)],  # 20 keypoints
        "skeleton": []
    }]
}

annotation_id = 1
image_global_id = 1  # Para evitar conflictos de ID entre archivos

def compute_bbox_from_keypoints(kpts, image_width, image_height, pad=10):
    points = np.array(kpts).reshape(-1, 3)
    visible = points[:, 2] > 0
    if not np.any(visible):
        return [0, 0, image_width, image_height]
    x_coords = points[visible, 0]
    y_coords = points[visible, 1]
    x_min = max(0, x_coords.min() - pad)
    y_min = max(0, y_coords.min() - pad)
    x_max = min(image_width, x_coords.max() + pad)
    y_max = min(image_height, y_coords.max() + pad)
    return [float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)]

# === PROCESAR TODOS LOS ARCHIVOS XML ===
xml_files = list(xml_folder_path.glob("*.xml"))

for xml_file in tqdm(xml_files, desc="Procesando archivos XML"):
    with open(xml_file, "rb") as f:
        tree = etree.parse(f)

    images = tree.xpath("//image")

    for img_elem in images:
        local_img_id = int(img_elem.attrib["id"])
        img_name = img_elem.attrib["name"]
        width = int(img_elem.attrib["width"])
        height = int(img_elem.attrib["height"])

        tag_elem = img_elem.find("tag")
        label = tag_elem.attrib["label"] if tag_elem is not None else "unknown"

        coco_output["images"].append({
            "id": image_global_id,
            "file_name": img_name,
            "width": width,
            "height": height,
            "label": label
        })

        kpts = [0] * 20 * 3
        num_keypoints = 0

        for point in img_elem.xpath("points"):
            label_point = point.attrib["label"]
            if label_point.isdigit() and 1 <= int(label_point) <= 20:
                idx = int(label_point) - 1
                x, y = map(float, point.attrib["points"].split(","))
                kpts[idx * 3: idx * 3 + 3] = [x, y, 2]
                num_keypoints += 1

        if num_keypoints > 0:
            bbox = compute_bbox_from_keypoints(kpts, width, height, padding)
            coco_output["annotations"].append({
                "id": annotation_id,
                "image_id": image_global_id,
                "category_id": 1,
                "keypoints": kpts,
                "num_keypoints": num_keypoints,
                "bbox": bbox,
                "area": bbox[2] * bbox[3],
                "iscrowd": 0
            })
            annotation_id += 1

        image_global_id += 1

# === GUARDAR JSON UNIFICADO ===
with open(output_path, "w") as f:
    json.dump(coco_output, f, indent=2)

print(f"Listo: JSON unificado guardado en {output_path}")


Procesando archivos XML: 100%|██████████| 12/12 [00:00<00:00, 27.68it/s]


Listo: JSON unificado guardado en /data/Datasets/Fish/CavFish/fisk_all_20kpts_2025.json


In [ ]:
###
#Se extrae la etiqueta y ruta de imagenes de 20 keypoints para 1758 spp. a partir de xml (CVAT)
###

import json
import numpy as np
from lxml import etree
from pathlib import Path
from tqdm import tqdm

# === CONFIGURA TUS RUTAS AQUÍ ===
xml_folder_path = Path("/data/Datasets/Fish/CavFish/xml/")
images_base_path = Path("/data/Datasets/Fish/CavFish/")  # Base donde están las subcarpetas de imágenes
output_path = Path("/data/Datasets/Fish/CavFish/fisk_all_20kpts_2025.json")
padding = 10  # píxeles extra para las bboxes

# === INICIALIZAR ESTRUCTURA COCO ===
coco_output = {
    "images": [],
    "annotations": [],
    "categories": [{
        "id": 1,
        "name": "fish",
        "keypoints": [str(i) for i in range(1, 21)],
        "skeleton": []
    }]
}

annotation_id = 1
image_global_id = 1

def compute_bbox_from_keypoints(kpts, image_width, image_height, pad=10):
    points = np.array(kpts).reshape(-1, 3)
    visible = points[:, 2] > 0
    if not np.any(visible):
        return [0, 0, image_width, image_height]
    x_coords = points[visible, 0]
    y_coords = points[visible, 1]
    x_min = max(0, x_coords.min() - pad)
    y_min = max(0, y_coords.min() - pad)
    x_max = min(image_width, x_coords.max() + pad)
    y_max = min(image_height, y_coords.max() + pad)
    return [float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)]

# === PROCESAR TODOS LOS ARCHIVOS XML ===
xml_files = list(xml_folder_path.glob("*.xml"))

for xml_file in tqdm(xml_files, desc="Procesando XML"):
    with open(xml_file, "rb") as f:
        tree = etree.parse(f)

    images = tree.xpath("//image")

    for img_elem in images:
        local_img_id = int(img_elem.attrib["id"])
        img_name = img_elem.attrib["name"]
        width = int(img_elem.attrib["width"])
        height = int(img_elem.attrib["height"])

        # Buscar ruta real dentro de cualquier subcarpeta de la base
        found = list(images_base_path.rglob(img_name))
        if not found:
            print(f"⚠️ Imagen no encontrada: {img_name}")
            continue

        relative_path = found[0].relative_to(images_base_path)  # ruta con subcarpeta incluida

        tag_elem = img_elem.find("tag")
        label = tag_elem.attrib["label"] if tag_elem is not None else "unknown"

        coco_output["images"].append({
            "id": image_global_id,
            "file_name": str(relative_path),  # ← AQUÍ va con subcarpeta
            "width": width,
            "height": height,
            "label": label
        })

        kpts = [0] * 20 * 3
        num_keypoints = 0

        for point in img_elem.xpath("points"):
            label_point = point.attrib["label"]
            if label_point.isdigit() and 1 <= int(label_point) <= 20:
                idx = int(label_point) - 1
                x, y = map(float, point.attrib["points"].split(","))
                kpts[idx * 3: idx * 3 + 3] = [x, y, 2]
                num_keypoints += 1

        if num_keypoints > 0:
            bbox = compute_bbox_from_keypoints(kpts, width, height, padding)
            coco_output["annotations"].append({
                "id": annotation_id,
                "image_id": image_global_id,
                "category_id": 1,
                "keypoints": kpts,
                "num_keypoints": num_keypoints,
                "bbox": bbox,
                "area": bbox[2] * bbox[3],
                "iscrowd": 0
            })
            annotation_id += 1

        image_global_id += 1

# === GUARDAR JSON UNIFICADO ===
with open(output_path, "w") as f:
    json.dump(coco_output, f, indent=2)

print(f"✅ JSON con rutas relativas guardado en: {output_path}")


Procesando XML: 100%|██████████| 12/12 [00:05<00:00,  2.38it/s]


✅ JSON con rutas relativas guardado en: /data/Datasets/Fish/CavFish/fisk_all_20kpts_2025.json


#### Dividir datos: entrenamiento y validacion

In [14]:
import json
import random
from pathlib import Path

# Load full annotation file (Compressed body as example)
input_path = Path("/data/Datasets/Fish/CavFish/fisk_all_20kpts_2025.json")
with input_path.open("r") as f:
    data = json.load(f)

# Shuffle and split
random.seed(42)
image_ids = list({img["id"] for img in data["images"]})
random.shuffle(image_ids)
split_index = int(0.9 * len(image_ids))
train_ids = set(image_ids[:split_index])
val_ids = set(image_ids[split_index:])

# Build separate image/annotation lists
def split_coco(data, selected_ids):
    return {
        "info": data.get("info", {}),
        "licenses": data.get("licenses", []),
        "images": [img for img in data["images"] if img["id"] in selected_ids],
        "annotations": [ann for ann in data["annotations"] if ann["image_id"] in selected_ids],
        "categories": data["categories"]
    }

train_data = split_coco(data, train_ids)
val_data = split_coco(data, val_ids)

# Save split files
train_path = "/data/Datasets/Fish/CavFish/fish20kpt_all_train.json"
val_path = "/data/Datasets/Fish/CavFish/fish20kpt_all_val.json"
with open(train_path, "w") as f:
    json.dump(train_data, f, indent=2)
with open(val_path, "w") as f:
    json.dump(val_data, f, indent=2)

train_path, val_path

('/data/Datasets/Fish/CavFish/fish20kpt_all_train.json',
 '/data/Datasets/Fish/CavFish/fish20kpt_all_val.json')

# Training _ HRNet

Modificar td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train.py en funcion de :

1. Numero de keypoints
2. Otras modificaciones de parametrso segun sea el caso. Fase 02.

Nota: .py fue modificado unicamnete en funcion de numero de kp.  

In [ ]:
!cd /home/arumota_pupils/Josue/1PECES/MMPose/mmpose


!python tools/train.py \
  /home/arumota_pupils/Josue/1PECES/MMPose/mmpose/configs/animal_2d_keypoint/topdown_heatmap/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train20.py 
  
#!export PYTHONPATH=$(pwd) && python tools/train.py configs/animal_2d_keypoint/topdown_heatmap/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train20.py 

# Inferencia_HRNet

Inferencia considerando:

1. Una imagen
2. Varias imagenes
3. Metrcias de evaluacion como RSME, MAE, R2, ...

In [2]:
#Organizar las imagenes de validacion de acuerdo a la division de val(10%). Se mota el json de val como ebtrada oara buscar las imagebnes. Esto evita crear una carpeta con imagens de val
import os
import json
import subprocess

# Config paths
json_path = '/data/Datasets/Fish/CavFish/fish20kpt_all_val.json'
dataset_root = '/data/Datasets/Fish/CavFish'  # Donde están las subcarpetas con las imágenes
output_folder = '/data/Datasets/Fish/CavFish/inference_20kpt/'
config_path = 'configs/animal_2d_keypoint/topdown_heatmap/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train20.py'
checkpoint_path = '/data/Josue/weights/Fish/HRNet_20kpt/best_PCK_epoch_110.pth'
merged_json_path = os.path.join(output_folder, 'all_keypoints_predicted20.json')

# Crear carpeta de salida
os.makedirs(output_folder, exist_ok=True)

# Leer JSON
with open(json_path, 'r') as f:
    data = json.load(f)

# Acumular predicciones
all_predictions = []

# Inferencia por imagen
for image_info in data['images']:
    file_name = image_info['file_name']  # Ruta relativa dentro de CavFish/
    full_path = os.path.join(dataset_root, file_name)

    img_name = os.path.basename(full_path)
    out_base = os.path.join(output_folder, img_name)
    out_json = out_base.replace('.jpg', '_keypoints.json').replace('.png', '_keypoints.json')

    if not os.path.exists(full_path):
        print(f"⚠️ Imagen no encontrada: {full_path}")
        continue

    cmd = [
        "python", "demo/image_demo.py",
        full_path,
        config_path,
        checkpoint_path,
        "--out-file", out_base,
        "--draw-heatmap",
        "--show-kpt-idx"
    ]

    print(f"🔁 Procesando {file_name}...")
    subprocess.run(cmd)

    if os.path.exists(out_json):
        with open(out_json, 'r') as f:
            pred = json.load(f)
            all_predictions.append(pred)
    else:
        print(f"⚠️ No se encontró JSON para {img_name}")

# Guardar predicciones consolidadas
with open(merged_json_path, 'w') as f:
    json.dump(all_predictions, f, indent=4)

print(f"\n✅ JSON consolidado guardado en: {merged_json_path}")



🔁 Procesando 2018 Guayavero Duda/IMG_0084-Edit.jpg...
Loads checkpoint by local backend from path: /data/Josue/weights/Fish/HRNet_20kpt/best_PCK_epoch_110.pth


✅ DEBUG: Patched model.dataset_meta keys = dict_keys(['dataset_name', 'num_keypoints', 'keypoint_names', 'keypoint_id2name', 'keypoint_name2id', 'skeleton', 'keypoint_colors', 'skeleton_links_color', 'flip_pairs', 'flip_indices', 'upper_body_ids', 'lower_body_ids', 'dataset_keypoint_weights', 'sigmas', 'CLASSES']) 


📍 Coordenadas predichas:
 - 0: x=296.3, y=1798.1, score=0.957

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/CavFish/inference_20kpt/IMG_0084-Edit_keypoints.json
 - 1: x=4150.8, y=1987.0, score=0.960

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/CavFish/inference_20kpt/IMG_0084-Edit_keypoints.json
 - 2: x=1617.8, y=1440.0, score=0.969

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/CavFish/inference_20kpt/IMG_0084-Edit_keypoints.json
 - 3: x=2253.4, y=2432.6, score=0.901

📝 Coordenadas guardada

In [ ]:
# Se estrae la ruta de la imagen. USado en HRNET final
import os
import json
import subprocess

# Configuración de rutas
json_path = '/data/Datasets/Fish/CavFish/fish20kpt_all_val.json'
dataset_root = '/data/Datasets/Fish/CavFish'
output_folder = '/data/Datasets/Fish/CavFish/inference_20kpt_HRNET2/'
config_path = 'configs/animal_2d_keypoint/topdown_heatmap/ak/td-hm_hrnet-w32_8xb32-300e_animalkingdom_P3_fish-256x256_train20.py'
checkpoint_path = '/data/Josue/weights/Fish/HRNet_20kpt/best_PCK_epoch_300.pth'
merged_json_path = os.path.join(output_folder, 'all_keypoints_predicted20.json')

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Leer el archivo JSON de validación
with open(json_path, 'r') as f:
    val_data = json.load(f)

# Almacenar todas las predicciones
all_predictions = []

# Recorrer cada imagen del archivo de validación
for image_info in val_data['images']:
    relative_path = image_info['file_name']  # ej: "2018 Guayavero Duda/IMG_0084-Edit.jpg"
    full_path = os.path.join(dataset_root, relative_path)

    # Nombre de imagen para guardar outputs
    img_name = os.path.basename(relative_path)
    out_base = os.path.join(output_folder, img_name)
    out_json = out_base.replace('.jpg', '_keypoints.json').replace('.png', '_keypoints.json')

    # Verificación de existencia de imagen
    if not os.path.exists(full_path):
        print(f"⚠️ Imagen no encontrada: {full_path}")
        continue

    # Ejecutar la inferencia
    cmd = [
        "python", "demo/image_demo.py",
        full_path,
        config_path,
        checkpoint_path,
        "--out-file", out_base,
        "--draw-heatmap",
        "--show-kpt-idx"
    ]

    print(f"🔁 Procesando {relative_path}...")
    subprocess.run(cmd)

    # Leer predicción y asociarla a la ruta relativa original (no solo nombre)
    if os.path.exists(out_json):
        with open(out_json, 'r') as f:
            pred = json.load(f)

        # Normalizar output para que coincida con GT
        pred["image"] = relative_path  # Sobrescribe con la ruta relativa del GT
        all_predictions.append(pred)
    else:
        print(f"⚠️ No se encontró JSON de salida para {img_name}")

# Guardar todas las predicciones en un solo archivo
with open(merged_json_path, 'w') as f:
    json.dump(all_predictions, f, indent=4)

print(f"\n✅ JSON consolidado guardado en: {merged_json_path}")


#### Metricas evaluacion modelo_HRNet

In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Rutas
pred_file = Path("/data/Datasets/Fish/CavFish/inference_20kpt_HRNET2/all_keypoints_predicted20_.json")
gt_file = Path("/data/Datasets/Fish/CavFish/fish20kpt_all_val.json")

# Cargar archivos
with pred_file.open("r") as f:
    pred_data = json.load(f)

with gt_file.open("r") as f:
    gt_data = json.load(f)

# Crear índice de anotaciones por imagen
image_id_to_name = {img['id']: img['file_name'] for img in gt_data['images']}
annotations_by_name = {
    image_id_to_name[ann['image_id']]: np.array(ann['keypoints']).reshape(-1, 3)[:, :2]
    for ann in gt_data['annotations']
}

# Comparación punto a punto
rows = []
gt_all = []
pred_all = []

for item in pred_data:
    name = item["image"]
    pred_kpts = {kp["name"]: (kp["x"], kp["y"]) for kp in item["keypoints"]}
    gt_kpts = annotations_by_name.get(name)

    if gt_kpts is None:
        print(f"⚠️ Imagen {name} no encontrada en GT.")
        continue

    for idx, (gt_xy, (kpt_name, pred_xy)) in enumerate(zip(gt_kpts, pred_kpts.items())):
        error = np.linalg.norm(np.array(pred_xy) - np.array(gt_xy))
        rows.append({
            "image": name,
            "keypoint": kpt_name,
            "gt_x": gt_xy[0],
            "gt_y": gt_xy[1],
            "pred_x": pred_xy[0],
            "pred_y": pred_xy[1],
            "euclidean_error": error
        })
        gt_all.extend(gt_xy)
        pred_all.extend(pred_xy)

df = pd.DataFrame(rows)

# --- 1. Métricas globales por keypoint ---
trait_metrics = []
for kpt in df["keypoint"].unique():
    sub = df[df["keypoint"] == kpt]
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    trait_metrics.append({
        "keypoint": kpt,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy)
    })
df_trait = pd.DataFrame(trait_metrics)
df_trait.to_csv("/data/Datasets/Fish/CavFish/metrics_per_keypoint20.csv", index=False)

# --- 2. Métricas por imagen ---
image_metrics = []
for img in df["image"].unique():
    sub = df[df["image"] == img]
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    image_metrics.append({
        "image": img,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy)
    })
df_image = pd.DataFrame(image_metrics)
df_image.to_csv("/data/Datasets/Fish/CavFish/metrics20_per_image.csv", index=False)

# --- 3. Métricas por keypoint e imagen ---
trait_image_metrics = []
for (img, kpt), sub in df.groupby(["image", "keypoint"]):
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    trait_image_metrics.append({
        "image": img,
        "keypoint": kpt,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy) if len(gt_xy) > 1 else None
    })
df_trait_image = pd.DataFrame(trait_image_metrics)
df_trait_image.to_csv("/data/Datasets/Fish/CavFish/metrics_per_keypoint20_image.csv", index=False)

print("✅ ¡Listo! Se guardaron los tres archivos CSV:")
print("- metrics_per_keypoint.csv")
print("- metrics_per_image.csv")
print("- metrics_per_keypoint_image.csv")

✅ ¡Listo! Se guardaron los tres archivos CSV:
- metrics_per_keypoint.csv
- metrics_per_image.csv
- metrics_per_keypoint_image.csv


In [2]:
df = pd.DataFrame(rows)

# Exportar datos individuales de puntos
df.to_csv("/data/Datasets/Fish/CavFish/keypoints_predictions_gt_vs_pred.csv", index=False)


# VitPose

## Training script

Must be copied on a terminal without the ! mark

In [ ]:
!cd /home/arumota_pupils/Josue/1PECES/MMPose/mmpose
    
!python tools/train.py \
    /home/arumota_pupils/Josue/1PECES/MMPose/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py

05/06 09:59:52 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 819089920
    GPU 0: NVIDIA GeForce RTX 4090
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.1, V12.1.105
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.2.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-genc

# Inferencia_ViTPose

Inferencia considerando:

1. Una imagen
2. Varias imagenes
3. Metrcias de evaluacion como RSME, MAE, R2, ...

In [1]:
# Se estrae la ruta de la imagen. Final
import os
import json
import subprocess

# Configuración de rutas
json_path = '/data/Datasets/Fish/CavFish/fish20kpt_all_val.json'
dataset_root = '/data/Datasets/Fish/CavFish'
output_folder = '/data/Datasets/Fish/CavFish/inference_20kpt_ViTPose/'
config_path = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py'
checkpoint_path = '/data/Josue/weights/Fish/ViTPose_20kpt/epoch_300.pth'
merged_json_path = os.path.join(output_folder, 'all_keypoints_predicted20_ViTPose.json')

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Leer el archivo JSON de validación
with open(json_path, 'r') as f:
    val_data = json.load(f)

# Almacenar todas las predicciones
all_predictions = []

# Recorrer cada imagen del archivo de validación
for image_info in val_data['images']:
    relative_path = image_info['file_name']  # ej: "2018 Guayavero Duda/IMG_0084-Edit.jpg"
    full_path = os.path.join(dataset_root, relative_path)

    # Nombre de imagen para guardar outputs
    img_name = os.path.basename(relative_path)
    out_base = os.path.join(output_folder, img_name)
    out_json = out_base.replace('.jpg', '_keypoints.json').replace('.png', '_keypoints.json')

    # Verificación de existencia de imagen
    if not os.path.exists(full_path):
        print(f"⚠️ Imagen no encontrada: {full_path}")
        continue

    # Ejecutar la inferencia
    cmd = [
        "python", "demo/image_demo.py",
        full_path,
        config_path,
        checkpoint_path,
        "--out-file", out_base,
        "--draw-heatmap",
        "--show-kpt-idx"
    ]

    print(f"🔁 Procesando {relative_path}...")
    subprocess.run(cmd)

    # Leer predicción y asociarla a la ruta relativa original (no solo nombre)
    if os.path.exists(out_json):
        with open(out_json, 'r') as f:
            pred = json.load(f)

        # Normalizar output para que coincida con GT
        pred["image"] = relative_path  # Sobrescribe con la ruta relativa del GT
        all_predictions.append(pred)
    else:
        print(f"⚠️ No se encontró JSON de salida para {img_name}")

# Guardar todas las predicciones en un solo archivo
with open(merged_json_path, 'w') as f:
    json.dump(all_predictions, f, indent=4)

print(f"\n✅ JSON consolidado guardado en: {merged_json_path}")


🔁 Procesando 2018 Guayavero Duda/IMG_0084-Edit.jpg...
Loads checkpoint by local backend from path: /data/Josue/weights/Fish/ViTPose_20kpt/epoch_300.pth


✅ DEBUG: Patched model.dataset_meta keys = dict_keys(['dataset_name', 'num_keypoints', 'keypoint_names', 'keypoint_id2name', 'keypoint_name2id', 'skeleton', 'keypoint_colors', 'skeleton_links_color', 'flip_pairs', 'flip_indices', 'upper_body_ids', 'lower_body_ids', 'dataset_keypoint_weights', 'sigmas', 'CLASSES']) 


📍 Coordenadas predichas:
 - 0: x=310.4, y=1807.8, score=0.941

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/CavFish/inference_20kpt_ViTPose/IMG_0084-Edit_keypoints.json
 - 1: x=4144.5, y=1963.1, score=0.941

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/CavFish/inference_20kpt_ViTPose/IMG_0084-Edit_keypoints.json
 - 2: x=1643.5, y=1441.8, score=0.944

📝 Coordenadas guardadas en JSON: /data/Datasets/Fish/CavFish/inference_20kpt_ViTPose/IMG_0084-Edit_keypoints.json
 - 3: x=2227.8, y=2409.4, score=0.997

📝 Coo

#### Metricas evaluacion modelo_ViTPose

In [2]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Rutas
pred_file = Path("/data/Datasets/Fish/CavFish/inference_20kpt_ViTPose/all_keypoints_predicted20_ViTPose.json")
gt_file = Path("/data/Datasets/Fish/CavFish/fish20kpt_all_val.json")

# Cargar archivos
with pred_file.open("r") as f:
    pred_data = json.load(f)

with gt_file.open("r") as f:
    gt_data = json.load(f)

# Crear índice de anotaciones por imagen
image_id_to_name = {img['id']: img['file_name'] for img in gt_data['images']}
annotations_by_name = {
    image_id_to_name[ann['image_id']]: np.array(ann['keypoints']).reshape(-1, 3)[:, :2]
    for ann in gt_data['annotations']
}

# Comparación punto a punto
rows = []
gt_all = []
pred_all = []

for item in pred_data:
    name = item["image"]
    pred_kpts = {kp["name"]: (kp["x"], kp["y"]) for kp in item["keypoints"]}
    gt_kpts = annotations_by_name.get(name)

    if gt_kpts is None:
        print(f"⚠️ Imagen {name} no encontrada en GT.")
        continue

    for idx, (gt_xy, (kpt_name, pred_xy)) in enumerate(zip(gt_kpts, pred_kpts.items())):
        error = np.linalg.norm(np.array(pred_xy) - np.array(gt_xy))
        rows.append({
            "image": name,
            "keypoint": kpt_name,
            "gt_x": gt_xy[0],
            "gt_y": gt_xy[1],
            "pred_x": pred_xy[0],
            "pred_y": pred_xy[1],
            "euclidean_error": error
        })
        gt_all.extend(gt_xy)
        pred_all.extend(pred_xy)

df = pd.DataFrame(rows)

# --- 1. Métricas globales por keypoint ---
trait_metrics = []
for kpt in df["keypoint"].unique():
    sub = df[df["keypoint"] == kpt]
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    trait_metrics.append({
        "keypoint": kpt,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy)
    })
df_trait = pd.DataFrame(trait_metrics)
df_trait.to_csv("/data/Datasets/Fish/CavFish/metrics_per_keypoint20_ViTPose.csv", index=False)

# --- 2. Métricas por imagen ---
image_metrics = []
for img in df["image"].unique():
    sub = df[df["image"] == img]
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    image_metrics.append({
        "image": img,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy)
    })
df_image = pd.DataFrame(image_metrics)
df_image.to_csv("/data/Datasets/Fish/CavFish/metrics20_per_image_ViTPose.csv", index=False)

# --- 3. Métricas por keypoint e imagen ---
trait_image_metrics = []
for (img, kpt), sub in df.groupby(["image", "keypoint"]):
    gt_xy = sub[["gt_x", "gt_y"]].values
    pred_xy = sub[["pred_x", "pred_y"]].values
    trait_image_metrics.append({
        "image": img,
        "keypoint": kpt,
        "MAE": mean_absolute_error(gt_xy, pred_xy),
        "RMSE": np.sqrt(mean_squared_error(gt_xy, pred_xy)),
        "R2": r2_score(gt_xy, pred_xy) if len(gt_xy) > 1 else None
    })
df_trait_image = pd.DataFrame(trait_image_metrics)
df_trait_image.to_csv("/data/Datasets/Fish/CavFish/metrics_per_keypoint20_image_ViTPose.csv", index=False)

print("✅ ¡Listo! Se guardaron los tres archivos CSV:")
print("- metrics_per_keypoint.csv")
print("- metrics_per_image.csv")
print("- metrics_per_keypoint_image.csv")

✅ ¡Listo! Se guardaron los tres archivos CSV:
- metrics_per_keypoint.csv
- metrics_per_image.csv
- metrics_per_keypoint_image.csv


In [3]:
df = pd.DataFrame(rows)

# Exportar datos individuales de puntos
df.to_csv("/data/Datasets/Fish/CavFish/keypoints_predictions_gt_vs_pred_ViTPose.csv", index=False)

In [4]:
df

,image,keypoint,gt_x,gt_y,pred_x,pred_y,euclidean_error
0,2018 Guayavero Duda/IMG_0084-Edit.jpg,0,348.2,1755.0,310.434174,1807.784528,64.903498
1,2018 Guayavero Duda/IMG_0084-Edit.jpg,1,4233.7,1981.7,4144.529692,1963.123710,91.084699
2,2018 Guayavero Duda/IMG_0084-Edit.jpg,2,1580.3,1440.7,1643.458887,1441.832249,63.169035
3,2018 Guayavero Duda/IMG_0084-Edit.jpg,3,2373.6,2434.5,2227.764995,2409.383272,147.982089
4,2018 Guayavero Duda/IMG_0084-Edit.jpg,4,730.6,1516.1,674.633837,1532.418608,58.296727
...,...,...,...,...,...,...,...
3515,2018 Yungillo/IMG_1771-Editar.jpg,15,1109.2,2313.9,1151.557112,2278.367850,55.287057
3516,2018 Yungillo/IMG_1771-Editar.jpg,16,945.4,1516.8,964.304492,1598.892157,84.240738
3517,2018 Yungillo/IMG_1771-Editar.jpg,17,2954.2,473.9,2967.150386,479.404474,14.071664
3518,2018 Yungillo/IMG_1771-Editar.jpg,18,2644.3,2054.0,2668.278506,2051.621838,24.096149


# Subset count per fish kind

In [3]:
import json
from collections import Counter

# Load your COCO JSON
with open("/data/Datasets/Fish/CavFish/fish20kpt_all_train.json", "r") as f:
    data = json.load(f)

# Extract labels from the images array
labels = [img.get("label") for img in data["images"] if "label" in img]

# Count occurrences of each label
label_counts = Counter(labels)

# Print results
print("Unique labels:", list(label_counts.keys()))
print("Counts per label:")
for label, count in label_counts.items():
    print(f"  {label}: {count}")

import json
from collections import Counter

# Load your COCO JSON
with open("/data/Datasets/Fish/CavFish/fish20kpt_all_val.json", "r") as f:
    data = json.load(f)

# Extract labels from the images array
labels = [img.get("label") for img in data["images"] if "label" in img]

# Count occurrences of each label
label_counts = Counter(labels)

# Print results
print("Unique labels:", list(label_counts.keys()))
print("Counts per label:")
for label, count in label_counts.items():
    print(f"  {label}: {count}")



Unique labels: ['Compressed body', 'Depressed body', 'Compressed body_without caudal fin', 'Rounded body', 'Fusiform', 'unknown']
Counts per label:
  Compressed body: 1005
  Depressed body: 518
  Compressed body_without caudal fin: 46
  Rounded body: 4
  Fusiform: 6
  unknown: 3
Unique labels: ['Compressed body', 'Depressed body', 'Compressed body_without caudal fin', 'Fusiform']
Counts per label:
  Compressed body: 121
  Depressed body: 47
  Compressed body_without caudal fin: 7
  Fusiform: 1


# end

## Inference over weights

In [ ]:
# 1. La imagen a inferir (Ruta)
# 2. Configuración usada para entrenar (Ruta)
# 3. Ruta de los pesos

!python demo/image_demo.py \
    "/data/Datasets/Fish/2022/images/val_fish/DSC03012_10032023194229.jpg" \
    "configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py" \
    "/data/Josue/weights/Fish/epoch_final_vit.pth" \
    --out-file "work_dirs/images/test_file.png" \
    --draw-heatmap \
    --show-kpt-idx

: 

Inference over a set of image definied over the input_folder path

In [ ]:
import os
import glob

# Define paths
input_folder = '/data/Datasets/Fish/2022/images/val_fish/'
output_folder = '/data/Datasets/Fish/2022/output/'
config_path = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py'
checkpoint_path = '/data/Josue/weights/Fish/epoch_final_vit.pth'

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get all image files (jpg/png/jpeg)
image_files = sorted(glob.glob(os.path.join(input_folder, '*.[jp][pn]g')))

# Loop through and run demo script
for img_path in image_files:
    img_name = os.path.basename(img_path)
    out_file = os.path.join(output_folder, img_name)

    !python demo/image_demo.py \
        "{img_path}" \
        "{config_path}" \
        "{checkpoint_path}" \
        --out-file "{out_file}" \
        --draw-heatmap \
        --show-kpt-idx

In [ ]:
# Generate Predictions
!python tools/test.py \
    configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py \
    /data/Josue/weights/Fish/epoch_600.pth \
    --dump /data/Josue/weights/Fish/predictions.pkl

In [ ]:
from mmpose.apis import inference_topdown, init_model
from mmengine.dataset import Compose
from mmcv import imread
import os
import glob
import json

# --- CONFIGURACIONES ---
config_path = 'configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain.py'
checkpoint_path = '/data/Josue/weights/Fish/epoch_final_vit.pth'
input_folder = '/data/Datasets/Fish/2022/images/val_fish/'
output_json = '/data/Datasets/Fish/2022/predicted_keypoints.json'

# --- INICIALIZAR MODELO ---
model = init_model(config_path, checkpoint_path, device='cuda:0')

# --- PROCESAR TODAS LAS IMÁGENES ---
image_files = sorted(glob.glob(os.path.join(input_folder, '*.[jp][pn]g')))
results = {}

for img_path in image_files:
    img = imread(img_path)
    h, w, _ = img.shape
    detection_result = [{'bbox': [0, 0, w, h]}]  # Asume una caja por imagen

    pose_results = inference_topdown(model, img, detection_result)

    if pose_results:
        keypoints = pose_results[0]['keypoints']  # (num_kpts, 3)
        results[os.path.basename(img_path)] = keypoints.tolist()

# --- GUARDAR ---
with open(output_json, 'w') as f:
    json.dump(results, f, indent=4)

print(f"Keypoints guardados en: {output_json}")


In [ ]:
import json
import random
import numpy as np
import matplotlib.pyplot as plt
from mmengine import load
from PIL import Image
from pathlib import Path

# Rutas a archivos
val_json_path = Path("/data/Datasets/Fish/2022/fish8kpt_val.json")
results_path = Path("/data/Josue/weights/Fish/predictions.pkl")

# Cargar anotaciones COCO y resultados
with open(val_json_path, "r") as f:
    val_data = json.load(f)
results = load(results_path)

# Mapeo de image_id a nombre de archivo
id_to_filename = {img["id"]: img["file_name"] for img in val_data["images"]}
image_dir = val_json_path.parent / "images"

# Escoger una muestra aleatoria
sample = random.choice(results)
img_id = sample.metainfo['img_id']  # ✅ acceder al atributo correctamente
img_name = id_to_filename[img_id]
img_path = image_dir / img_name
img = Image.open(img_path)

# Obtener keypoints de la primera instancia
kpts = sample.pred_instances.keypoints

# Visualizar
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis("off")

for x, y in kpts[0]:
    if 0 <= x <= img.width and 0 <= y <= img.height:
        plt.scatter(x, y, c='red', s=40)
        plt.text(x + 2, y + 2, f"{int(x)},{int(y)}", fontsize=8, color='yellow')

plt.title(f"Predicted Keypoints – {img_name}")
plt.show()


In [ ]:
import json
import numpy as np
from lxml import etree
from pathlib import Path
from tqdm import tqdm

# === CONFIGURA TUS RUTAS AQUÍ ===
xml_input_path = Path("/data/Datasets/Fish/2022/2022_2023_General.xml")
output_path = Path("/data/Datasets/Fish/2022/fisk_8kpts.json")
padding = 10  # píxeles extra para las bboxes

# === PARSEAR XML ===
with open(xml_input_path, "rb") as f:
    tree = etree.parse(f)

images = tree.xpath("//image")

# === INICIALIZAR ESTRUCTURA COCO ===
coco_output = {
    "images": [],
    "annotations": [],
    "categories": [{
        "id": 1,
        "name": "fish",
        "keypoints": [str(i) for i in range(1, 9)],
        "skeleton": []
    }]
}

annotation_id = 1

def compute_bbox_from_keypoints(kpts, image_width, image_height, pad=10):
    points = np.array(kpts).reshape(-1, 3)
    visible = points[:, 2] > 0
    if not np.any(visible):
        return [0, 0, image_width, image_height]
    x_coords = points[visible, 0]
    y_coords = points[visible, 1]
    x_min = max(0, x_coords.min() - pad)
    y_min = max(0, y_coords.min() - pad)
    x_max = min(image_width, x_coords.max() + pad)
    y_max = min(image_height, y_coords.max() + pad)
    return [float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)]

# === PROCESAR IMÁGENES Y ANOTACIONES ===
for img_elem in tqdm(images):
    img_id = int(img_elem.attrib["id"])
    img_name = img_elem.attrib["name"]
    width = int(img_elem.attrib["width"])
    height = int(img_elem.attrib["height"])

    coco_output["images"].append({
        "id": img_id,
        "file_name": img_name,
        "width": width,
        "height": height
    })

    kpts = [0] * 8 * 3
    num_keypoints = 0

    for point in img_elem.xpath("points"):
        label = point.attrib["label"]
        if label.isdigit() and 1 <= int(label) <= 8:
            idx = int(label) - 1
            x, y = map(float, point.attrib["points"].split(","))
            kpts[idx * 3: idx * 3 + 3] = [x, y, 2]
            num_keypoints += 1

    if num_keypoints > 0:
        bbox = compute_bbox_from_keypoints(kpts, width, height, padding)
        coco_output["annotations"].append({
            "id": annotation_id,
            "image_id": img_id,
            "category_id": 1,
            "keypoints": kpts,
            "num_keypoints": num_keypoints,
            "bbox": bbox,
            "area": bbox[2] * bbox[3],
            "iscrowd": 0
        })
        annotation_id += 1

# === GUARDAR JSON ===
with open(output_path, "w") as f:
    json.dump(coco_output, f, indent=2)

print(f"Listo: {output_path}")


In [ ]:
import json
import random
import numpy as np
import matplotlib.pyplot as plt
from mmengine import load
from PIL import Image
from pathlib import Path

# Cargar el archivo val.json
val_json_path = Path("/data/Datasets/Fish/2022/fish8kpt_val.json")
with open(val_json_path, "r") as f:
    val_data = json.load(f)

# Cargar las predicciones
results = load("/data/Josue/weights/Fish/epoch_600.pth")  # Asegúrate que este path es correcto o reemplázalo

# Mapear image_id a file_path
id_to_filename = {img["id"]: img["file_name"] for img in val_data["images"]}
image_dir = Path(val_json_path).parent  # /data/Datasets/Fish/2022

# Seleccionar una predicción aleatoria que esté en el val set
val_ids = set(id_to_filename.keys())
sample = next(r for r in results if r.img_id in val_ids)

# Obtener keypoints predichos
kpts = sample.pred_instances.keypoints  # (K, 3)
img_path = image_dir / id_to_filename[sample.img_id]
img = Image.open(img_path)

# Mostrar imagen con keypoints
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.axis("off")

for x, y, v in kpts:
    if v > 0:
        plt.scatter(x, y, c='red', s=40)
        plt.text(x + 2, y + 2, f"{int(x)},{int(y)}", fontsize=8, color='yellow')

plt.title(f"Predicted Keypoints for Image ID: {sample.img_id}")
plt.show()


In [ ]:
!python demo/image_demo.py \
  /data/Datasets/Fish/FIB/FIB/inference/IMG_0181-Editar.jpg \
  configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etest.py \
  work_dirs/td-hm_ViTPose-fish9_8xb32-100etrain/epoch_600.pth \
  --out-file vis_results/vitpose_600epoch/fish_infer_0181.jpg \
  --draw-heatmap \
  --show-kpt-idx

In [ ]:
!cd /home/arumota_pupils/Josue/1PECES/MMPose/mmpose
    
!python tools/train.py \
    /home/arumota_pupils/Josue/1PECES/MMPose/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain_loss.py

In [ ]:
!cd /home/arumota_pupils/Josue/1PECES/MMPose/mmpose
    

!python tools/train.py \
    /home/arumota_pupils/Josue/1PECES/MMPose/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_ViTPose-fish9_8xb32-100etrain_loss_logMSE.py